In [ ]:
# ===== CONFIG =====
DATA_CSV = "radiomics_features_2D_with_AnnotationDetails.csv"
IMG_ROOT = "/home/24chuong.ta/anaconda3/chun/DICOM 1-100/Image_Encoder_Outputs_Swin"  # <-- adjust if needed

ID_COL     = "ID"            # optional, for reporting
STEM_COL   = "__stem__"      # points to image embedding
LABEL_COL  = "Subtype"       # target label

# Agreed radiomics list (must match CSV headers)
SELECTED_RADIOMICS = [
    "Gross size",
    "her2 expression cd",
    "original_shape2D_Elongation",
    "original_shape2D_Perimeter",
    "original_shape2D_MinorAxisLength",
    "original_shape2D_MajorAxisLength",
    "original_shape2D_Sphericity",
    "original_shape2D_MeshSurface",
    "original_glrlm_GrayLevelNonUniformity",
    "original_glrlm_RunLengthNonUniformity",
    "original_glszm_LargeAreaEmphasis",
    "original_glszm_LargeAreaLowGrayLevelEmphasis",
    "original_glszm_ZoneVariance",
    "original_glszm_ZonePercentage",
    "original_gldm_DependenceNonUniformity",
    "original_gldm_DependenceNonUniformityNormalized",
    "original_gldm_DependenceVariance",
]

# Treat these textual columns as categorical for one-hot (add/remove as needed)
FORCE_CATEGORICAL = [
    "BI-RAD", "Mass Shape", "Mass Margin", "Diagnosis",
    "Histologic grade", "Calcification morphology", "Gross Feature"
]

# Labels to drop
DROP_LABELS = {"Unidentifiable", "Unknown", "", "NA", "nan", "None", "N/A"}

TEST_SIZE    = 0.20
RANDOM_SEED  = 42


In [ ]:
# ===== IMPORTS =====
import os, re, glob
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler, normalize
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, confusion_matrix,
    precision_recall_curve, average_precision_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

np.random.seed(RANDOM_SEED)


In [ ]:
# ===== LOAD & CLEAN =====
df = pd.read_csv(DATA_CSV)

def clean_label(x):
    if pd.isna(x): return ""
    return str(x).replace("\u00A0"," ").replace("\u200B","").strip()

df[LABEL_COL] = df[LABEL_COL].apply(clean_label)
before = len(df)
df = df[~df[LABEL_COL].isin(DROP_LABELS)].reset_index(drop=True)
after = len(df)
print(f"Dropped {before - after} rows by label filter; remaining: {after}")

# Columns present
cols = df.columns.tolist()

# 1) Radiomics (use only the agreed list if present)
radiomics_cols = [c for c in SELECTED_RADIOMICS if c in df.columns]
missing_rads   = [c for c in SELECTED_RADIOMICS if c not in df.columns]
if missing_rads:
    print("[warn] Missing radiomics (ignored):", missing_rads)
print(f"Using {len(radiomics_cols)} radiomics:", radiomics_cols)

# 2) Identify candidates excluding label/ID/stem and the selected radiomics
exclude = set([LABEL_COL, ID_COL, STEM_COL]) | set(radiomics_cols)
candidate_cols = [c for c in cols if c not in exclude]

# We will:
# - send object-dtype or forced columns to categorical one-hot
# - send numeric columns to 'other numeric' UNLESS they look like extra radiomics:
#     any column starting with 'original_' that is NOT in our agreed radiomics list is EXCLUDED
forced_set = set(FORCE_CATEGORICAL)
cat_cols, num_extra_cols = [], []
skipped_extra_original = []

for c in candidate_cols:
    if c in forced_set:
        cat_cols.append(c)
        continue

    if df[c].dtype == "O":  # object/string -> categorical
        cat_cols.append(c)
        continue

    # numeric path
    if c.startswith("original_") and (c not in radiomics_cols):
        # looks like an extra radiomics feature not in the agreed list -> skip
        skipped_extra_original.append(c)
        continue

    # keep other numeric (non-radiomics) columns
    num_extra_cols.append(c)

numeric_cols = radiomics_cols + num_extra_cols

print(f"Additional numeric (non-radiomics): {len(num_extra_cols)}")
print(f"Categorical/text (one-hot): {len(cat_cols)}")
print(f"Total numeric used: {len(numeric_cols)}")

if skipped_extra_original:
    print(f"Excluded extra 'original_*' columns not in SELECTED_RADIOMICS: {len(skipped_extra_original)}")
    # Uncomment to inspect names:
    # print(skipped_extra_original[:20])

# Class names (fixed order)
class_names = np.unique(df[LABEL_COL].astype(str).values)
C = len(class_names)
print("Classes:", class_names.tolist())


In [ ]:
# ===== SPLIT =====
y_all = pd.Categorical(df[LABEL_COL].astype(str).values, categories=class_names).codes
sss = StratifiedShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=RANDOM_SEED)
tr_idx, te_idx = next(sss.split(df, y_all))

df_tr = df.iloc[tr_idx].reset_index(drop=True)
df_te = df.iloc[te_idx].reset_index(drop=True)

y_tr = pd.Categorical(df_tr[LABEL_COL].astype(str).values, categories=class_names).codes
y_te = pd.Categorical(df_te[LABEL_COL].astype(str).values, categories=class_names).codes

print(f"Train n={len(df_tr)} | Test n={len(df_te)}")


In [ ]:
# ===== TABULAR PREPROCESSOR =====
num_pipe = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="median")),
    ("sc",  StandardScaler(with_mean=True, with_std=True)),
])

cat_pipe = Pipeline(steps=[
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

pre = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

X_tab_tr = pre.fit_transform(df_tr[numeric_cols + cat_cols])
X_tab_te = pre.transform(df_te[numeric_cols + cat_cols])

feat_names = pre.get_feature_names_out().tolist()
print("Tabular shapes → train:", X_tab_tr.shape, " test:", X_tab_te.shape)


In [ ]:
# ===== IMAGE RESOLVER (RAW) =====
def stem_to_paths(stem: str):
    """
    e.g., '1062443_0062_LCC' ->
    case_dir = '1062443_0062'
    cropped  = IMG_ROOT/1062443_0062/1062443_0062_LCC_cropped
    """
    if not isinstance(stem, str) or "_" not in stem:
        return None, None
    parts = stem.split("_")
    if len(parts) < 3:
        return None, None
    case_dir = "_".join(parts[:2])
    view     = parts[2]
    cropped_dir = os.path.join(IMG_ROOT, case_dir, f"{case_dir}_{view}_cropped")
    return case_dir, cropped_dir

def find_emb_file(cropped_dir: str):
    if not cropped_dir or not os.path.isdir(cropped_dir): return None
    p = Path(cropped_dir)
    # prefer npy
    npys = sorted(glob.glob(str(p / "*crop_emb.npy")))
    if npys: return npys[0]
    csvs = sorted(glob.glob(str(p / "*crop_emb.csv")) + glob.glob(str(p / "*crop_emb.CSV")))
    if csvs: return csvs[0]
    return None

def load_embedding(fpath: str):
    if fpath is None: return None
    if fpath.lower().endswith(".npy"):
        v = np.load(fpath).astype(float).reshape(-1)
    else:
        v = pd.read_csv(fpath, header=None).to_numpy().astype(float).reshape(-1)
    return v

def build_image_matrix(df_split):
    vecs, has_img = [], []
    files = []
    for stem in df_split[STEM_COL].astype(str).tolist():
        _, cropped = stem_to_paths(stem)
        f = find_emb_file(cropped)
        files.append(f)
        if f is None:
            vecs.append(None); has_img.append(0.0)
        else:
            v = load_embedding(f)
            vecs.append(v);   has_img.append(1.0)
    # common width by max length
    max_dim = max((len(v) for v in vecs if isinstance(v, np.ndarray)), default=0)
    X_raw = np.zeros((len(df_split), max_dim), dtype=float)
    for i, v in enumerate(vecs):
        if isinstance(v, np.ndarray):
            d = min(max_dim, len(v))
            X_raw[i, :d] = v[:d]
    # row-wise L2 norm (safe; no leakage)
    X_raw = normalize(X_raw, norm="l2", axis=1) if max_dim > 0 else X_raw
    has_flag = np.array(has_img, dtype=np.float32).reshape(-1, 1)
    return X_raw, has_flag, files

X_img_raw_tr, has_img_tr, files_tr = build_image_matrix(df_tr)
X_img_raw_te, has_img_te, files_te = build_image_matrix(df_te)

print("Image shapes → train:", X_img_raw_tr.shape, " test:", X_img_raw_te.shape)
print("Image coverage → train:", float(has_img_tr.mean()), " test:", float(has_img_te.mean()))


In [ ]:
# ===== FUSE =====
def fuse(tab, img_raw, has_flag):
    parts = [tab]
    if img_raw is not None and img_raw.shape[1] > 0: parts.append(img_raw)
    if has_flag is not None: parts.append(has_flag)
    return np.hstack(parts)

X_tr = fuse(X_tab_tr, X_img_raw_tr, has_img_tr)
X_te = fuse(X_tab_te, X_img_raw_te, has_img_te)

print("Fused dims → train:", X_tr.shape, " test:", X_te.shape)


In [ ]:
# ===== RANDOM FOREST =====
weights = compute_class_weight("balanced", classes=np.arange(C), y=y_tr)
class_weight = {i: w for i, w in enumerate(weights)}

rf = RandomForestClassifier(
    n_estimators=800,
    max_depth=None,
    min_samples_leaf=1,
    min_samples_split=2,
    max_features="sqrt",
    class_weight=class_weight,
    n_jobs=-1,
    random_state=RANDOM_SEED,
)

rf.fit(X_tr, y_tr)
proba_te = rf.predict_proba(X_te)

# Align to full class set (if any class missing in train)
proba_full = np.zeros((len(y_te), C), dtype=float)
proba_full[:, rf.classes_] = proba_te
y_pred = proba_full.argmax(1)

acc  = accuracy_score(y_te, y_pred)
f1m  = f1_score(y_te, y_pred, average="macro")
f1u  = f1_score(y_te, y_pred, average="micro")
f1w  = f1_score(y_te, y_pred, average="weighted")
print(f"[Tabular + Image(raw)] Acc={acc:.3f} | F1(macro)={f1m:.3f} | F1(micro)={f1u:.3f} | F1(weighted)={f1w:.3f}")

print("\nClassification report:")
print(classification_report(
    y_te, y_pred,
    labels=np.arange(C), target_names=class_names,
    digits=3, zero_division=0
))


In [ ]:
# ===== CONFUSION MATRIX =====
def show_confusion_matrix(y_true, y_hat, class_names, title="Confusion Matrix", normalize=False):
    cm = confusion_matrix(y_true, y_hat, labels=np.arange(len(class_names)))
    disp = cm.astype(float) / cm.sum(axis=1, keepdims=True).clip(min=1) if normalize else cm
    print(f"{title} (rows=true, cols=pred):\n", cm)
    fig, ax = plt.subplots(figsize=(6,5), dpi=140)
    ax.imshow(disp, interpolation="nearest")
    ax.set_xticks(np.arange(len(class_names))); ax.set_xticklabels(class_names, rotation=45, ha="right")
    ax.set_yticks(np.arange(len(class_names))); ax.set_yticklabels(class_names)
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_title(title + (" (normalized)" if normalize else ""))
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            val = f"{disp[i,j]:.2f}" if normalize else f"{cm[i,j]}"
            ax.text(j, i, val, ha="center", va="center", fontsize=8)
    plt.tight_layout(); plt.show()
    return cm

_ = show_confusion_matrix(y_te, y_pred, class_names, title="Confusion Matrix — Tabular + Image(raw)", normalize=False)

# ===== PER-CLASS PR CURVES =====
Y_bin = pd.get_dummies(pd.Series(y_te), columns=range(C)).values
ap_per_class = {}
fig, ax = plt.subplots(figsize=(7,5), dpi=140)
for k, name in enumerate(class_names):
    precision, recall, _ = precision_recall_curve(Y_bin[:, k], proba_full[:, k])
    ap = average_precision_score(Y_bin[:, k], proba_full[:, k])
    ap_per_class[name] = ap
    ax.plot(recall, precision, label=f"{name} (AP={ap:.2f})")
ax.set_xlabel("Recall"); ax.set_ylabel("Precision"); ax.grid(True, alpha=0.3)
ax.set_title("Per-class PR — Tabular + Image(raw)")
ax.legend(fontsize=8); plt.tight_layout(); plt.show()

pd.DataFrame([{
    "variant": "tabular_plus_image_raw",
    "n_features": X_tr.shape[1],
    "acc": acc, "f1_macro": f1m, "f1_micro": f1u, "f1_weighted": f1w,
    **{f"AP_{k}": v for k, v in ap_per_class.items()}
}])
